In [ ]:
# File path relative to basedir
dir_ = 'data/manta/2017-01-09/'
fname = dir_ + 'dsDNA_-8TA+10NTD_1Dto40A_1000x_532nm_600mW_after_5_min_second_2.dat'

# Static configuration

In [ ]:
from pathlib import Path

In [ ]:
remote_origin_basedir = '/mnt/Antonio/'    # Remote dir containing the original acquisition data
temp_basedir = '/mnt/ramdisk/'             # Local temp dir with very fast access 
local_archive_basedir = '/mnt/archive/'    # Local dir for archiving data
remote_archive_basedir = '/mnt/wAntonio/'  # Remote dir for archiving data

In [ ]:
# Identity data used when the metadata does not contain identity info:
default_identity = dict(
    author='Antonino Ingargiola',
    author_affiliation='UCLA',
    creator='Antonino Ingargiola',
    creator_affiliation='UCLA')

# Imports

In [ ]:
from pathlib import Path
import time
import numpy as np
import tables
import yaml
from tqdm import tqdm_notebook, tnrange  # progress bar

import phconvert as phc
phc.__version__

In [ ]:
import niconverter as nic
nic.__version__

# Resolve paths

In [ ]:
source_filename = Path(temp_basedir, fname)
if not source_filename.is_file():
    source_filename = Path(remote_origin_basedir, fname)
assert source_filename.is_file()
source_filename

In [ ]:
out_path = Path(temp_basedir, fname).with_suffix('.hdf5')
out_path

In [ ]:
out_path = Path(out_path.parent, out_path.stem + '_inplace.hdf5')
out_path

In [ ]:
# Create the output data folder if necessary
out_path.parent.mkdir(parents=True, exist_ok=True)

# Load metadata

In [ ]:
meta_filename = source_filename.with_suffix('.yml')
assert meta_filename.is_file()

In [ ]:
with open(meta_filename) as f:
    metadata = yaml.load(f)
metadata

Default measurement type is PAX when not specified:

In [ ]:
measurement_type = metadata.pop('measurement_type', 'PAX')

# Preprocess timestamps

In [ ]:
t_start = time.time()

In [ ]:
h5file, meta = nic.ni96ch_process_spots(source_filename, out_path=out_path, close=True)
ts_unit = 1 / meta['clock_frequency']
fname, meta

In [ ]:
time.time() - t_start

In [ ]:
h5file = tables.open_file(str(out_path), mode='a')
dt, endianess, meta = nic.detectformat(source_filename)
ts_unit = 1 / meta['clock_frequency']
t2, aem2 = nic.get_photon_data_arr(h5file, spots=np.arange(48))
meta['acquisition_duration'] = nic.duration(t2, ts_unit)
source_filename.stem, meta

In [ ]:
time.time() - t_start

In [ ]:
print('Measurement duration: %.1f s' %  meta['acquisition_duration'])
print('Transfer rate: %.2f MB/s' % (meta['num_timestamps'] / meta['acquisition_duration'] * 4 / (1024**2)))

In [ ]:
neg_jumps = []
for tx in tqdm_notebook(t2):
    d = np.diff(tx[:])
    assert (d >= 0).all()
    neg_jumps.append(d[d<0])
    
missed_ov_jumps = [len(j) for j in neg_jumps]
for ch, jumps in enumerate(missed_ov_jumps):
    if jumps > 0:
        print('CH%d: %s' % (ch, jumps))

In [ ]:
time.time() - t_start

# Create Photon-HDF5

## Populate metadata

In [ ]:
if 'identity' not in metadata:
    metadata['identity'] = default_identity
metadata

In [ ]:
metadata = nic.populate_metadata_smFRET_48spots(metadata, source_filename, 
                                                acq_duration=meta['acquisition_duration'])
metadata

## Create the Photon-HDF5 file

In [ ]:
str(source_filename)

In [ ]:
h5file.filename

In [ ]:
data = nic.fill_photon_data_tables(metadata.copy(), h5file, ts_unit,
                                   measurement_type=measurement_type)

In [ ]:
[f for f in data.keys() if not f.startswith('photon')]

In [ ]:
%%timeit -n1 -r1
phc.hdf5.save_photon_hdf5(data, h5file=h5file, overwrite=True, close=True)

In [ ]:
#%debug

In [ ]:
time.time() - t_start